# PART 3

## BUILDING SUMMARY TABLES

In this section, I build a single SQLite Database via python. It contains three tables:
1. Sales by date by hour
- by calendar date (YYYY-MM-DD) and hour of the day, determines the total spend in the store, the number of transactions, and a count of the number of items. 
2. Sales by owner by year by month
- contains the following columns: card_no, year, month, sales, transactions, and items
3. Sales by product description by year by month
- contains the following columns: upc, description, department number, department name, year, month, sales, transactions, and items

In [ ]:
import sqlite3
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
from pandas.io import gbq

In [ ]:
# define table exists function (which will be used later)
def tbl_exists(client, table_ref):
    from google.cloud.exceptions import NotFound
    try:
        client.get_table(table_ref)
        return True
    except NotFound:
        return False

In [ ]:
# establish connection with GBQ
# set up credentials
service_path = "/Users/austinsmith/Documents/Fall21/ADA/Wedge"
service_file = '/thewedge-austin-4c5ad634b17b.json' 
gbq_proj_id = 'thewedge-austin'
gbq_dataset_id = 'the_wedge_A' 

private_key =service_path + service_file

credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# establish connection with gbq
client = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [ ]:
# create a local database
db = sqlite3.connect("thewedge_part3.db")

In [ ]:
# create a cursor to talk to the database
cur = db.cursor()

In [ ]:
# the queries will be done in Google Big Query, then exported to textfiles, and uploaded to the database
# this script will create the tables that those files will populate

# These columns are needed for the first table

cur.execute('''DROP TABLE IF EXISTS sales_by_date_hour''')
cur.execute('''CREATE TABLE sales_by_date_hour (
    calendar_day TEXT,
    hour TEXT,
    spend FLOAT,
    transactions INTEGER,
    items INTEGER)''')

db.commit()

In [ ]:
my_table = "sales_by_date_hour"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name) 
    #note to self: also clean table out!

# NEED TO COMMENT THESE OUT IF CREATING THE TABLE FOR THE FIRST TIME
    query_text ="".join(['DELETE FROM `',table_full_name,'` WHERE 1=1'])
    # you have to have WHERE clause in a DELETE for GBQ

    job_config = bigquery.QueryJobConfig()

    query_job = client.query(
        query_text,
        location="US",
        job_config=job_config,
    )  # API request - starts the query

    query_job.result()  # Waits for the query to finish

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
SELECT
  FORMAT_DATE('%F',datetime) AS Calendar_Day,
  EXTRACT(hour FROM datetime) AS hour,
  ROUND(SUM(total),2) AS Spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS Transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS Items_Count
FROM
  `thewedge-austin.the_wedge_A.clean_transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  Calendar_Day,
  hour
ORDER BY
  hour
"""

query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))

In [ ]:
# put the results into a pandas dataframe and write it to a local text file

df = client.query(sql).to_dataframe()

df.to_csv('sales_by_date_hour.txt', sep= '\t')

In [ ]:
# then write the text file to the database

input_file = 'sales_by_date_hour.txt'

with open(input_file,'r',encoding="Latin-1") as ifile :
    next(ifile)  
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO sales_by_date_hour (calendar_day, hour, spend, transactions, items)
            VALUES (?,?,?,?,?)''', line) 

db.commit()


In [ ]:
# code for tables 2 and 3 mimics table 1

cur.execute('''DROP TABLE IF EXISTS owner_sales_by_yearmonth''')
cur.execute('''CREATE TABLE owner_sales_by_yearmonth (
    card_no INTEGER,
    year TIMESTAMP,
    month TIMESTAMP,
    sales FLOAT,
    transactions INTEGER,
    items INTEGER)''')

db.commit()

In [ ]:
my_table = "owner_sales_by_yearmonth"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name) 
    #note to self: also clean table out!

# NEED TO COMMENT THESE OUT IF CREATING THE TABLE FOR THE FIRST TIME
    query_text ="".join(['DELETE FROM `',table_full_name,'` WHERE 1=1'])
    # you have to have WHERE clause in a DELETE for GBQ

    job_config = bigquery.QueryJobConfig()

    query_job = client.query(
        query_text,
        location="US",
        job_config=job_config,
    )  # API request - starts the query

    query_job.result()  # Waits for the query to finish

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
SELECT
  card_no,
  EXTRACT(YEAR from datetime) AS year,
  EXTRACT(MONTH from datetime) AS month,
  ROUND(SUM(total),2) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `thewedge-austin.the_wedge_A.clean_transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  card_no,
  year,
  month
ORDER BY
  card_no,
  year,
  month
"""

query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))

In [ ]:
# put the results into a pandas dataframe and write it to a local text file

df = client.query(sql).to_dataframe()

df.to_csv('owner_sales_by_yearmonth.txt', sep= '\t')

In [ ]:
input_file = 'owner_sales_by_yearmonth.txt' 

with open(input_file,'r',encoding="Latin-1") as ifile :
    next(ifile) # 
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split("\t")
        cur.execute('''
            INSERT INTO owner_sales_by_yearmonth (card_no, year, month, sales, transactions, items)
            VALUES (?,?,?,?,?,?)''', line) 
db.commit()

In [ ]:
# code for table 3

cur.execute('''DROP TABLE IF EXISTS product_description_by_yearmonth''')
cur.execute('''CREATE TABLE product_description_by_yearmonth (
    upc INTEGER,
    description TEXT,
    department_number INTEGER,
    department_description TEXT,
    year DATETIME,
    month DATETIME,
    sales FLOAT,
    transactions INTEGER,
    items INTEGER)''')

db.commit()

In [ ]:
my_table = "product_description_by_yearmonth"

table_full_name = ".".join([gbq_proj_id,gbq_dataset_id,my_table])

if not tbl_exists(client, table_full_name) :
    table_ref = client.create_table(
        table = table_full_name
    )
else :
    table_ref = client.get_table(table_full_name) 
    #note to self: also clean table out!

# # NEED TO COMMENT THESE OUT IF CREATING THE TABLE FOR THE FIRST TIME
    query_text ="".join(['DELETE FROM `',table_full_name,'` WHERE 1=1'])
    # you have to have WHERE clause in a DELETE for GBQ

    job_config = bigquery.QueryJobConfig()

    query_job = client.query(
        query_text,
        location="US",
        job_config=job_config,
    )  # API request - starts the query

    query_job.result()  # Waits for the query to finish

job_config = bigquery.QueryJobConfig(destination=table_ref)

sql = """
SELECT
  upc,
  description,
  department AS department_number,
  CASE
    WHEN department = 1 THEN 'Packaged Grocery'
    WHEN department = 2 THEN 'Product'
    WHEN department = 3 THEN 'Bulk'
    WHEN department = 4 THEN 'Ref Grocery'
    WHEN department = 5 THEN 'Cheese'
    WHEN department = 6 THEN 'Frozen'
    WHEN department = 7 THEN 'Bread'
    WHEN department = 8 THEN 'Deli'
    WHEN department = 9 THEN 'Gen Merch'
    WHEN department = 10 THEN 'Supplements'
    WHEN department = 11 THEN 'Personal Care'
    WHEN department = 12 THEN 'Herbs & Spices'
    WHEN department = 13 THEN 'Meat'
    WHEN department = 14 THEN 'Juice Bar'
    WHEN department = 16 THEN 'Fish & Seafood'
    WHEN department = 17 THEN 'Bakehouse'
    WHEN department = 18 THEN 'Flowers'
    WHEN department = 19 THEN 'WedgeWorldWide'
    WHEN department = 20 THEN 'Misc P/I WWW'
    WHEN department = 21 THEN 'Catering'
    WHEN department = 22 THEN 'Beer & Wine'
  ELSE
  'not described'
END
  AS department_description,
  EXTRACT(YEAR
  FROM
    datetime) AS year,
  EXTRACT(MONTH
  FROM
    datetime) AS month,
  ROUND(SUM(total),2) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `thewedge-austin.the_wedge_A.clean_transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  upc,
  description,
  department,
  year,
  month
ORDER BY
  year,
  month
"""

query_job = client.query(sql, job_config=job_config)  # Make an API request.
query_job.result()  # Wait for the job to complete.

print("Query results loaded to the table {}".format(table_ref.table_id))

In [ ]:
# put the results into a pandas dataframe and write it to a local text file

df = client.query(sql).to_dataframe()

df.to_csv('product_description_by_yearmonth.txt', sep= '\t')


In [ ]:
# this fills up the table with the transaction rows, skipping headers

input_file = 'product_description_by_yearmonth.txt' 

with open(input_file,'r',encoding="Latin-1") as ifile :
    next(ifile) # 
    for idx, line in enumerate(ifile.readlines()) :
        line = line.strip().split('\t')
        cur.execute('''
            INSERT INTO product_description_by_yearmonth (upc, description, department_number, department_description,
            year, month, sales, transactions, items)
            VALUES (?,?,?,?,?,?,?,?,?)''', line) 

        
#         if idx > 100 :
#             break # TAKE THIS OUT LATER!

db.commit()

## The sql queries that were used to generate Tables 1, 2, and 3 from above



1. Sales by date by hour

SELECT
  FORMAT_DATE('%F',datetime) AS Calendar_Day,
  EXTRACT(hour FROM datetime) AS hour,
  ROUND(SUM(total),2) AS Spend,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS Transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS Items_Count
FROM
  `thewedge-austin.the_wedge_A.clean_transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  Calendar_Day,
  hour
ORDER BY
  hour


2. Sales by owner by year by month

SELECT
  card_no,
  EXTRACT(YEAR from datetime) AS year,
  EXTRACT(MONTH from datetime) AS month,
  ROUND(SUM(total),2) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `thewedge-austin.the_wedge_A.clean_transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  card_no,
  year,
  month
ORDER BY
  card_no,
  year,
  month

3. Sales by product description by year by month

SELECT
  upc,
  description,
  department AS department_number,
  CASE
    WHEN department = 1 THEN 'Packaged Grocery'
    WHEN department = 2 THEN 'Product'
    WHEN department = 3 THEN 'Bulk'
    WHEN department = 4 THEN 'Ref Grocery'
    WHEN department = 5 THEN 'Cheese'
    WHEN department = 6 THEN 'Frozen'
    WHEN department = 7 THEN 'Bread'
    WHEN department = 8 THEN 'Deli'
    WHEN department = 9 THEN 'Gen Merch'
    WHEN department = 10 THEN 'Supplements'
    WHEN department = 11 THEN 'Personal Care'
    WHEN department = 12 THEN 'Herbs & Spices'
    WHEN department = 13 THEN 'Meat'
    WHEN department = 14 THEN 'Juice Bar'
    WHEN department = 16 THEN 'Fish & Seafood'
    WHEN department = 17 THEN 'Bakehouse'
    WHEN department = 18 THEN 'Flowers'
    WHEN department = 19 THEN 'WedgeWorldWide'
    WHEN department = 20 THEN 'Misc P/I WWW'
    WHEN department = 21 THEN 'Catering'
    WHEN department = 22 THEN 'Beer & Wine'
  ELSE
  'not described'
END
  AS department_description,
  EXTRACT(YEAR
  FROM
    datetime) AS year,
  EXTRACT(MONTH
  FROM
    datetime) AS month,
  ROUND(SUM(total),2) AS sales,
  COUNT(DISTINCT(DATE(datetime) || register_no || emp_no || trans_no)) AS transactions,
  SUM(CASE
      WHEN (trans_status = 'V' OR trans_status = 'R') THEN -1
    ELSE
    1
  END
    ) AS items
FROM
  `thewedge-austin.the_wedge_A.clean_transArchive_*`
WHERE
  department != 0
  AND department != 15
  AND trans_status != 'M'
  AND trans_status != 'C'
  AND trans_status != 'J'
  AND (trans_status = ''
    OR trans_status = ' '
    OR trans_status = 'V'
    OR trans_status = 'R')
GROUP BY
  upc,
  description,
  department,
  year,
  month
ORDER BY
  year,
  month
